# Домашнее задание №2: NER и коллокации

## Импорты

In [1]:
!pip3 install Wikipedia-API
!pip3 install spacy
!pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from collections import defaultdict
from string import punctuation
import re
import gzip
import gensim
from gensim.models import Phrases
import json
import wikipediaapi
import spacy
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tqdm

nltk.download('popular')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-dat

True

In [3]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

## Данные

In [ ]:
# !wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Musical_Instruments_5.json.gz/

--2022-12-16 23:40:48--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Musical_Instruments_5.json.gz/
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
ERROR: cannot verify jmcauley.ucsd.edu's certificate, issued by ‘CN=InCommon RSA Server CA,OU=InCommon,O=Internet2,L=Ann Arbor,ST=MI,C=US’:
  Unable to locally verify the issuer's authority.
To connect to jmcauley.ucsd.edu insecurely, use `--no-check-certificate'.


In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        rev = json.loads(l)
        rev_text = rev['reviewText']
        yield rev_text

In [ ]:
rev_gen = parse('Musical_Instruments_5.json.gz')

In [ ]:
for i in range(10):
    print('Review', i)
    print(next(rev_gen))
    print('\n\n')

Review 0
It's good for beginners



Review 1
I recommend this starter Ukulele kit.  I has everything you
need to learn the Ukulele.



Review 2
G'daughter received this for Christmas present last year and plays if often.



Review 3
According to my order history, I bought this this kit towards the end of 2012 as a Black Friday Week Lightning Deal. I'm writing this nearly four years later.

As with any stringed instrument, the strings really needed to stretch out before they would stay in tune. So, for the first several days, it was play a few minutes, tune, play a few minutes, tune. Then it was tuning daily. After a few weeks of this, the uke held a tune great. A tuner is not included in the package, which is the bulk of the minus star because a real beginner wouldn't have a way to tune the thing. (You need either a tuner or a reference note, say, on a piano or keyboard.) You can download a decent tuner app, though, for a buck or so. Do it, because you're going to need it a lot for the

Если рассмотреть несколько текстов отзывов, то становится понятно, что сам продукт вообще редко упоминают. Либо продукт заменяется местоимением, либо он упоминается где-то в сравнениях, либо с тональной характеристикой.

С точки зрения технической реализации есть следующие проблемы:
* Допустим, мы собрали словарь терминов, которые хотим выделять из текстов. Каким образом мы должны определить, какие из этих слов релевантнты конкретному тексту? Возможно, это придётся делать с помощью частотности или другой метрике релевантности слова тексту (статистика, косинусная близость с текстом)
* Названия продуктов часто не связаны с самими продуктами (например, *Chord Buddy* -- это гитара; хотя конечно *chord* что-то и рассказывает о музыке)

## О методах

### Написание правил

Так как у нас комплексная задача (найти именованные сущности + продукты), то кандидатов на роль именованных сущностей можно собирать двумя способами:
* Для именованных сущностей, то есть названий модели гитары, например, часто действует правило нескольких подряд токенов с заглавной буквы. При этом скорее всего во всём остальном тексте по такому же правилу будут определяться модели, с которыми аспект в отзыв сравнивается, либо если мы говорим конкретно про музыкальную область, то это будут имена исполнителей, которые играли на похожих инструментах и т.д. Кроме того иногда у моделей есть цифра в названии. Для такого рода правил можно использовать регулярные выражения или внутренние методы питона.
* Для выделения аспектов можно использовать синтаксические шаблоны, например, ими могут быть существительное или несколько подряд существительных, прилагательное + существительное, а также несколько существительных и цифра, а также некоторые парсеры умеют распознавать имена собственные (интересно, будет ли это работать для NE?). К такому роду правил можно прикрутить и проверку морфологии в том числе, например наличие генитива у следующего существительного (хотя скорее всего, в нашем задании такое правило не понадобится). Чтобы достать такие коллокации, нужно сделать морфосинтаксический парсинг и взять нужные н-граммы. Или взять RegExpTokenizer от `nltk`, написать ему грамматику, и вывести нужные коллокации.

__Достоинства__:
* Понятные методы
* Много чего выделится

__Недостатки__:
* Долго писать правила
* Можно описать не все правила
* Даже если опишем правила для синтаксиса, непонятно, хорошо ли будет выполнен сам морфосинтаксический парсинг
* RegExp токенизатор не умеет выделять накладывающиеся коллокации
* В принципе не так много решений, позволяющих писать правила для английского? Yargy не позволит сделать синтаксис, так как работает на OpenCorpora; Natasha тоже для русского. Правила будут слишком конкретными, что затруднит их написание. В основном всё на более сложных структурах написано (об этом ниже)


### Извлечение ключевых слов и коллокаций
На самом деле, товары в данном случае можно определить как ключевые слова текстов, так как они являются темами отзывов. В этом случае можно использовать следующие техники нахождения ключевых слов:
* Частотность
* Метрики: TF-IDF, Okapi BM25
* Статистические метрики: Хи-квадрат, log-likelihood ratio, weirdness
* Меры ассоциации: pmi, t-критерий Стьюдента и др. через `nltk.collocations`


Во всех этих случаях можно пробовать использовать предварительное выделение кандидатов (грамматика, синтаксис, просто убрать стоп-слова). После выделения кандидатов можно применить порог для определения самых важных слов и коллокаций.

__Достоинства__:
* Много чего выделится через частотность и через метрики, которые основаны на частоте
* Статистические методы основаны на важности и совместной встречаемости слов, что позволит выделить самые релевантные вещи, а не всё подряд
* `nltk.collocations` удобно выделяет и кандидатов, и метрики считает, можно выбирать сразу столько кандидатов, сколько необходимо

__Недостатки__:
* Без предварительного выделения кандидатов даже в самые важные попадёт всё, что угодно
* Нужно искать порог для определения самых важных штук, потому что может попадаться мусор
* Нужна лемматизация (и возможно морфосинтаксический парсинг)

### Тезаурусы
В извлечении ключевых слов в какой-то области могут помочь базы знаний и тезаурусы. В отличие от всех остальных способов, здесь понадобится доступ к базе знаний, если для неё нет питоновской оболочки.

Какие базы знаний можно использовать?
* Википедия
* WordNet

Как это работает? Так как мы уже знаем с какой областью работаем, то ключевые слова и родственные им можно найти в базе знаний.
1. С помощью WordNet можно найти все гипонимы к музыкальным инструментам или похожим категориям -- это будут ключевые слова, которые можно использовать для поиска в текстах
2. Википедия может быть использована и для выделения ключевых слов, и для выделения сущностей, так как кроме базы понятий, она содержит знания о конкретных моделях -- их точно так же можно использовать для поиска по текстам

__Достоинства__:
* Без мусора, только релевантное
* Не нужно выделять предварительно кандидатов

__Недостатки__:
* Не факт, что Википедия знает обо всех конкретных моделях, тогда их нужно каким-то образом добирать (например, с помощью тех же самых правил или метаинформации о товарах)
* Могут быть проблемы и с абстрактными синонимами тоже (модель, инструмент и т.д.). Здесь возможное решение состоит в том, чтобы из собранных сущностей повыделять части (*музыкальный инструмент* -> *инструмент*)
* Нужна лемматизация

Помогут ли для расширения эмбеддинги? Возможно, но я не уверена. Это будет необходимо только если база знаний достаточно маленькая. В остальных случаях скорее всего будет добавляться мусор, но надо пробовать.

(Считаю тезаурусы в такой момент каким-то читерством)

### Использование готовых библиотек
Для NER и выделения ключевых слов существует несколько готовых библиотек с уже реализованными алгоритмами:
* NER: `spaCy`
* KE: `summa`, `RAKE`, `KeyBERT`, `PKE`

В их основе лежат различные алгоритмы выделения ключевых слов. Например, `summa` основана на TextRank, `KeyBERT`, как понятно из названия, работает на эмбеддингах.

Для нашей задачи эти решения, по всей видимости, придётся комбинировать.

__Достоинства__:
* Всё реализовано за нас
* В принципе хорошо работают
* spaCy можно обучать на своих данных

__Недостатки__:
* Неинтересно, потому что не надо ничего писать самим
* Неидеально

## Собираем словарь
Будем использовать метод с базами знаний: используем API википедии и WordNet, чтобы достать релевантные слова и коллокации, а затем расширим.

* Википедия

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('en')

In [ ]:
cat = wiki_wiki.page('List_of_musical_instruments')

In [ ]:
list(cat.links)

['20th-century classical music',
 '21st-century classical music',
 'A-flat clarinet',
 'A-side and B-side',
 'Accordina (instrument)',
 'Accordion',
 'Accordola',
 'Acoustic-electric guitar',
 'Acoustic bass guitar',
 'Acoustic guitar',
 'Adungu',
 'Aeolian harp',
 'Aesthetics of music',
 'Afoxé',
 'Agida',
 'Agogô',
 'Agung',
 'Air horn',
 'Ajaeng',
 'Akkordolia',
 'Alboka',
 'Album',
 'Album era',
 'Alfaia',
 'Algaita',
 'Algerian mandole',
 'All-female band',
 'Almpfeiferl',
 'AlphaSphere (instrument)',
 'Alphorn',
 'Alternative rock',
 'Alto clarinet',
 'Alto flute',
 'Alto horn',
 'Alto sarrusophone',
 'Alto saxophone',
 'Alto trombone',
 'Ancient Tamil music',
 'Ancient music',
 'Andalusi classical music',
 'Angklung',
 'Angélique (instrument)',
 'Apinti',
 'Appalachian dulcimer',
 'Arabic music',
 'Arbajo',
 'Archlute',
 'Archtop guitar',
 'Arghul',
 'Arobapá',
 'Arpeggione',
 'Art music',
 'Asadullah (instrument)',
 'Ashiko',
 'Atabaque',
 'Atenteben',
 'Audio engineer',
 'Audi

* Remove music
* Remove brackets

In [ ]:
instruments = [l.lower() for l in list(cat.links)]

instruments[:5]

['20th-century classical music',
 '21st-century classical music',
 'a-flat clarinet',
 'a-side and b-side',
 'accordina (instrument)']

In [ ]:
instruments = [i for i in instruments if 'music' not in i]

instruments[:5]

['a-flat clarinet',
 'a-side and b-side',
 'accordina (instrument)',
 'accordion',
 'accordola']

In [ ]:
instruments = [re.sub(' \(.+\)', '', i) for i in instruments]

instruments[:10]

['a-flat clarinet',
 'a-side and b-side',
 'accordina',
 'accordion',
 'accordola',
 'acoustic-electric guitar',
 'acoustic bass guitar',
 'acoustic guitar',
 'adungu',
 'aeolian harp']

In [ ]:
len(instruments)

890

Достать модели инструментов кажется не совсем возможным. У некоторых категорий есть графа с компаниями-изготовителями, при желании можно достать их, но это невозможно сделать через обёртку API. Однако опять же как мы поймём, где чьи? Видимо, нужно каким-то образом сохранять связь между например, сделать из списка словарь, где ключом будет инструмент, а значениями, если таковые существуют, изготовители.

In [ ]:
guitar = wiki_wiki.page('Guitar')

In [ ]:
guitar.backlinks

* WordNet

In [ ]:
mus_instrument_wn = wn.synsets('musical_instrument')[0]

mus_instrument_wn

Synset('musical_instrument.n.01')

In [ ]:
hyper = mus_instrument_wn.hypernyms()

hyper

[Synset('device.n.01')]

In [ ]:
hypo = mus_instrument_wn.hyponyms()

hypo

[Synset('barrel_organ.n.01'),
 Synset('bass.n.07'),
 Synset('calliope.n.02'),
 Synset('electronic_instrument.n.01'),
 Synset('jew's_harp.n.01'),
 Synset('keyboard_instrument.n.01'),
 Synset('music_box.n.01'),
 Synset('percussion_instrument.n.01'),
 Synset('stringed_instrument.n.01'),
 Synset('wind_instrument.n.01')]

In [ ]:
for i in range(len(hypo)):
    print(hypo[i], '---', hypo[i].hyponyms())

Synset('barrel_organ.n.01') --- []
Synset('bass.n.07') --- [Synset('bass_fiddle.n.01'), Synset('bass_guitar.n.01'), Synset('bass_horn.n.01'), Synset('bombardon.n.02')]
Synset('calliope.n.02') --- []
Synset('electronic_instrument.n.01') --- [Synset('electric_organ.n.01'), Synset('synthesizer.n.02'), Synset('theremin.n.01')]
Synset('jew's_harp.n.01') --- []
Synset('keyboard_instrument.n.01') --- [Synset('accordion.n.01'), Synset('celesta.n.01'), Synset('clavichord.n.01'), Synset('clavier.n.02'), Synset('organ.n.05'), Synset('piano.n.01'), Synset('synthesizer.n.02')]
Synset('music_box.n.01') --- []
Synset('percussion_instrument.n.01') --- [Synset('bones.n.01'), Synset('chime.n.01'), Synset('cymbal.n.01'), Synset('drum.n.01'), Synset('glockenspiel.n.01'), Synset('gong.n.01'), Synset('kettle.n.04'), Synset('lagerphone.n.01'), Synset('maraca.n.01'), Synset('marimba.n.01'), Synset('piano.n.01'), Synset('rain_stick.n.01'), Synset('steel_drum.n.01'), Synset('triangle.n.05'), Synset('vibraphone.

Поступим следующим образом: возьмём гипероним только на уровень выше, а гипонимы соберём все, которые найдём. Тогда:

In [ ]:
# гиперонимы
hyper = []
for s in mus_instrument_wn.hypernyms():
    s_lemmas = s.lemmas()
    for lemma in s_lemmas:
        h_lemma = ' '.join(str(lemma.name()).lower().split('_'))
        hyper.append(h_lemma)

hyper

['device']

In [ ]:
# гипонимы

high_hypo = mus_instrument_wn.hyponyms()

high_hypo

[Synset('barrel_organ.n.01'),
 Synset('bass.n.07'),
 Synset('calliope.n.02'),
 Synset('electronic_instrument.n.01'),
 Synset('jew's_harp.n.01'),
 Synset('keyboard_instrument.n.01'),
 Synset('music_box.n.01'),
 Synset('percussion_instrument.n.01'),
 Synset('stringed_instrument.n.01'),
 Synset('wind_instrument.n.01')]

In [ ]:
def all_hypo(sn):
    hypo = sn.hyponyms()
    if hypo:
        hypo_lemmas = []
        print('HYPO', hypo)
        for h in hypo:
            h_lemmas = h.lemmas()
            for lemma in h_lemmas:
                h_lemma = ' '.join(str(lemma.name()).lower().split('_'))
                hypo_lemmas.append(h_lemma)
        return hypo_lemmas

In [ ]:
hypo = []
for high_h in high_hypo:
    res = all_hypo(high_h)
    if res:
        hypo += res

HYPO [Synset('bass_fiddle.n.01'), Synset('bass_guitar.n.01'), Synset('bass_horn.n.01'), Synset('bombardon.n.02')]
HYPO [Synset('electric_organ.n.01'), Synset('synthesizer.n.02'), Synset('theremin.n.01')]
HYPO [Synset('accordion.n.01'), Synset('celesta.n.01'), Synset('clavichord.n.01'), Synset('clavier.n.02'), Synset('organ.n.05'), Synset('piano.n.01'), Synset('synthesizer.n.02')]
HYPO [Synset('bones.n.01'), Synset('chime.n.01'), Synset('cymbal.n.01'), Synset('drum.n.01'), Synset('glockenspiel.n.01'), Synset('gong.n.01'), Synset('kettle.n.04'), Synset('lagerphone.n.01'), Synset('maraca.n.01'), Synset('marimba.n.01'), Synset('piano.n.01'), Synset('rain_stick.n.01'), Synset('steel_drum.n.01'), Synset('triangle.n.05'), Synset('vibraphone.n.01')]
HYPO [Synset('banjo.n.01'), Synset('bowed_stringed_instrument.n.01'), Synset('chordophone.n.01'), Synset('clavichord.n.01'), Synset('clavier.n.02'), Synset('dulcimer.n.01'), Synset('guitar.n.01'), Synset('koto.n.01'), Synset('piano.n.01'), Synset('

In [ ]:
hypo

['bass fiddle',
 'bass viol',
 'bull fiddle',
 'double bass',
 'contrabass',
 'string bass',
 'bass guitar',
 'bass horn',
 'sousaphone',
 'tuba',
 'bombardon',
 'bombard',
 'electric organ',
 'electronic organ',
 'hammond organ',
 'organ',
 'synthesizer',
 'synthesiser',
 'theremin',
 'accordion',
 'piano accordion',
 'squeeze box',
 'celesta',
 'clavichord',
 'clavier',
 'klavier',
 'organ',
 'pipe organ',
 'piano',
 'pianoforte',
 'forte-piano',
 'synthesizer',
 'synthesiser',
 'bones',
 'castanets',
 'clappers',
 'finger cymbals',
 'chime',
 'bell',
 'gong',
 'cymbal',
 'drum',
 'membranophone',
 'tympan',
 'glockenspiel',
 'orchestral bells',
 'gong',
 'tam-tam',
 'kettle',
 'kettledrum',
 'tympanum',
 'tympani',
 'timpani',
 'lagerphone',
 'maraca',
 'marimba',
 'xylophone',
 'piano',
 'pianoforte',
 'forte-piano',
 'rain stick',
 'steel drum',
 'triangle',
 'vibraphone',
 'vibraharp',
 'vibes',
 'banjo',
 'bowed stringed instrument',
 'string',
 'chordophone',
 'clavichord',
 'c

In [ ]:
musical_instruments = set(instruments + hypo + hyper)

In [ ]:
len(musical_instruments)

941

Теперь проверяем есть ли они в наших текстах. Для этого сначала лемматизируем их:

In [ ]:
# откатить генератор на начальное состояние
# rev_gen = parse('Musical_Instruments_5.json.gz')

In [ ]:
# nlp = spacy.load('en_core_web_sm')

In [ ]:
# lemmatized_list = []
# i = 0
# for text in rev_gen:
#     doc = nlp(text)
#     lemmatized = []
#     for token in doc:
#         lemma = token.lemma_.lower()
#         if lemma not in PUNCTUATION:
#             if lemma not in ENG_SW:
#                 lemmatized.append(lemma)
#     lemmatized = ' '.join(lemmatized)
#     lemmatized_list.append(lemmatized)
#     if len(lemmatized_list) == 10:
#         i += 10
#         print(i)
#         with open('lemmatized wo stop-words reviews amazon mus instruments.txt', 'a', encoding='utf-8') as f:
#             f.write('\n\n------\n\n'.join(lemmatized_list))
#         lemmatized_list = []

In [ ]:
# или открыть готовый файл
with open('lemmatized wo stop-words reviews amazon mus instruments.txt', 'r', encoding='utf-8') as f:
    reviews = f.read().split('\n\n------\n\n')

In [ ]:
musical_instruments_ngrams = [tuple(ne.split()) for ne in musical_instruments]

musical_instruments_ngrams[:5]

[('octave', 'mandolin'),
 ('lusheng',),
 ('chapman', 'stick'),
 ('song',),
 ('clappers',)]

In [ ]:
ner_from_dict = set()
tokenized_reviews = []  # сохраняем токенизированные тексты
ngrams_reviews = []  # сохраняем нграммы из текстов

for rev in tqdm.tqdm(reviews):
    # убираем мусорные пробелы
    rev = re.sub('\s+', ' ', rev.lower())

    # токенизация пословная
    tokenized_rev = word_tokenize(rev)
    tokenized_reviews.append(tokenized_rev)

    # униграммы, биграммы, триграммы

    unigrams = set(tokenized_rev)
    bigrams = set(nltk.bigrams(tokenized_rev))
    trigrams = set(nltk.trigrams(tokenized_rev))

    ngrams_reviews.append([unigrams, bigrams, trigrams])

    for ne in musical_instruments_ngrams:
        new_ne = None
        if len(ne) == 1:
            if ne[0] in unigrams:
                new_ne = ne[0]
        if len(ne) > 1:
            if ne in bigrams or ne in trigrams:
                new_ne = ne
        if new_ne:
            if isinstance(new_ne, str):
                found_ne = new_ne
            else:
                found_ne = ' '.join(new_ne)
            if found_ne not in ner_from_dict:
                ner_from_dict.add(found_ne)

print(len(ner_from_dict))

100%|██████████| 208207/208207 [02:33<00:00, 1354.84it/s]

277


In [ ]:
with open('tokenized reviews.txt', 'w', encoding='utf-8') as f:
    f.write('\n\n------\n\n'.join(['\n'.join(rev) for rev in tokenized_reviews]))

In [ ]:
ner_from_dict

{'accordion',
 'acoustic bass guitar',
 'acoustic guitar',
 'album',
 'alternative rock',
 'alto clarinet',
 'alto flute',
 'alto saxophone',
 'alto trombone',
 'archtop guitar',
 'ashiko',
 'audio engineer',
 'aulos',
 'autoharp',
 'backup band',
 'bagpipe',
 'bajo sexto',
 'balalaika',
 'band',
 'banjo',
 'banjo uke',
 'baritone',
 'baritone guitar',
 'baritone saxophone',
 'baroque guitar',
 'baroque violin',
 'barrel drum',
 'bass banjo',
 'bass clarinet',
 'bass drum',
 'bass fiddle',
 'bass flute',
 'bass guitar',
 'bass horn',
 'bass pedals',
 'bass saxophone',
 'bass trombone',
 'bass trumpet',
 'bass violin',
 'bassoon',
 'bazooka',
 'beatboxing',
 'bell',
 'big band',
 'bluegrass mandolin',
 'blues',
 'blues harp',
 'bombard',
 'bones',
 'bongo drums',
 'boomwhacker',
 'bouzouki',
 'brass',
 'brass instrument',
 'bugle',
 'c melody saxophone',
 'cabasa',
 'cajun accordion',
 'carillon',
 'cavaquinho',
 'cello',
 'chalumeau',
 'chapman stick',
 'charango',
 'chime',
 'choir',


На самом деле, здесь заметно, что всё-таки выделяются и лишние вещи.

In [ ]:
len(ner_from_dict)

277

Отлично! Теперь попробуем расширить наш словарь. Получим ключевые н-граммы с помощью эмбеддингов. Интуиция следующая: если мы найдём векторы слов исходя из контекста (ещё было бы хорошо, чтобы модель умела распознавать неизвестные ей слова в том числе), то сможем найти похожие на музыкальные инструменты слова из текста, так как они будут употребляться в схожем контексте. Возможно, есть смысл сравнивать с любыми музыкальными инструментами, так как нам важны именно контексты, в которых они встречаются. Выбирать будем первых 3 кандидатов, так как мы отметили, что случаев реального упоминания товара крайне мало.

На самом деле, текстов достаточно много, так что мы можем обучить свою собственную модель

In [ ]:
full_text = []
for rev in reviews:
    rev = re.sub('\s+', ' ', rev.lower())
    sentences = sent_tokenize(rev)
    full_text += sentences

In [ ]:
len(full_text)

210329

In [ ]:
full_text[1]

'recommend starter ukulele kit everything need learn ukulele'

In [ ]:
with open('for model.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(full_text))

In [ ]:
f = 'for model.txt'
data = gensim.models.word2vec.LineSentence(f)
phrases = Phrases(data, min_count=1, threshold=1)

In [ ]:
model = gensim.models.Word2Vec(phrases[data], size=300, window=5, min_count=2)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
model.init_sims(replace=True)
model_path = "model.bin"

model.wv.save_word2vec_format(model_path, binary=True)

Смотрим сходство:

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [ ]:
ner_from_dict_vecs = {}
for instr in ner_from_dict:
    # проверить коллокацию сразу нельзя
    splited_instr = instr.split()
    if len(splited_instr) > 1:
        instr_vecs = []
        for i in splited_instr:
            if i in model:
                instr_vecs.append(model[i])
        if len(instr_vecs) > 1:
            ner_from_dict_vecs[instr] = np.array(instr_vecs).mean(axis=0)
        else:
            ner_from_dict_vecs[instr] = instr_vecs[0]
    else:
        if splited_instr[0] in model:
            ner_from_dict_vecs[splited_instr[0]] = model[splited_instr[0]]

In [ ]:
ner_from_dict_vecs['kazoo']

array([ 9.14176181e-02,  1.53432554e-02, -1.27983168e-01,  4.85348850e-02,
        1.04357526e-02,  1.04593158e-01,  7.22777247e-02, -1.94444976e-04,
        2.71386541e-02, -6.54474646e-02,  4.52086926e-02,  1.11690395e-01,
       -8.36484414e-03,  4.05251868e-02,  6.00370802e-02,  5.46973012e-02,
       -4.56930175e-02, -5.41922171e-03,  8.97701904e-02, -2.98418682e-02,
       -6.34310171e-02, -1.22706272e-01, -4.12986763e-02,  4.17748876e-02,
        2.92769372e-02,  4.14482206e-02, -2.48319358e-02,  4.99588959e-02,
        3.25732194e-02,  4.16288711e-02,  2.62486428e-04,  1.59622785e-02,
        2.14661565e-02,  1.11659365e-02, -5.03697284e-02, -5.31667992e-02,
       -7.05442578e-02, -3.31101455e-02, -4.47181575e-02,  4.81416192e-03,
        9.23014954e-02, -1.38544720e-02,  3.78281288e-02,  3.86178456e-02,
       -7.03576878e-02, -7.96595775e-03, -1.47384657e-02, -1.38688674e-02,
       -3.42422985e-02,  1.16511770e-01, -1.63746979e-02,  1.34706959e-01,
       -5.67535870e-02,  

In [ ]:
len(ner_from_dict_vecs)

244

In [ ]:
ner_words = list(ner_from_dict_vecs)
ner_vecs = np.array(list(ner_from_dict_vecs.values()))

In [ ]:
ner_vecs.shape

(244, 300)

In [ ]:
new_candidates = []
dict_vec = ner_vecs.mean(axis=0).reshape(1, -1)

for rev in tqdm.tqdm(ngrams_reviews):
    rev_info = {}

    # униграммы
    for unigram in rev[0]:
        if unigram in model:
            rev_info[unigram] = model[unigram]

    # биграммы
    for b in rev[1]:
        bigram = ' '.join(b)
        instr_vecs = []
        for i in b:
            if i in model:
                instr_vecs.append(model[i])
        if len(instr_vecs) > 1:
            rev_info[bigram] = np.array(instr_vecs).mean(axis=0)
        if len(instr_vecs) == 1:
            rev_info[bigram] = instr_vecs[0]

    # триграммы
    for t in rev[2]:
        trigram = ' '.join(t)
        instr_vecs = []
        for i in t:
            if i in model:
                instr_vecs.append(model[i])
        if len(instr_vecs) > 1:
            rev_info[trigram] = np.array(instr_vecs).mean(axis=0)
        if len(instr_vecs) == 1:
            rev_info[trigram] = instr_vecs[0]

    # сравнение
    text_words = np.array(list(rev_info))
    text_vecs = np.array(list(rev_info.values()))

    if len(text_vecs) >= 3:

        # допустим, нужная коллокация будет похожа на средний вектор от всех векторов слов из словаря
        scores = cosine_similarity(text_vecs, dict_vec)
        # ограничим скор, должен быть хотя бы больше 0.5
        if np.any(scores.ravel() > 0.5):
            sorted_scores_indx = np.argsort(scores, axis=0)[::-1]
            new_candidates.append(list(text_words[sorted_scores_indx.ravel()])[0])

100%|██████████| 208207/208207 [11:34<00:00, 299.90it/s]


In [ ]:
print(new_candidates[:5])

['beginner', 'starter', 'christmas present', 'quibble jordan play', 'self favor']


In [ ]:
final_ner = set(ner_words + new_candidates)

len(final_ner)

84412

Серьёзное увеличение:) Интересно, насколько много там нерелевантного и насколько много там того, что можно было бы объединить под одним названием..

In [ ]:
with open('final_ner.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(final_ner))

Непонятно, стоит ли сейчас объединять их. Попробуем найти релевантное без объединения, а затем посмотрим на то, как их можно объединить.

## Коллокации


Сначала отранжируем униграммы, триграммы и униграммы, а затем будем искать в них нужные, в конечном итоге достанем уже со скором.

In [4]:
with open('tokenized reviews.txt', 'r', encoding='utf-8') as f:
    tokenized_reviews = [rev.split('\n') for rev in f.read().split('\n\n------\n\n')]

In [5]:
len(tokenized_reviews)

208207

In [6]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = nltk.collocations.BigramCollocationFinder.from_documents(tokenized_reviews)
finder2.apply_freq_filter(2)  # вряд ли что-то будет часто повторяться

In [ ]:
scored2_pmi = finder2.score_ngrams(bigram_measures.pmi)

In [7]:
scored2_raw_freq = finder2.score_ngrams(bigram_measures.raw_freq)

In [ ]:
scored2_likelihood_ratio = finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [12]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder3 = nltk.collocations.TrigramCollocationFinder.from_documents(tokenized_reviews)
finder3.apply_freq_filter(2)

In [ ]:
scored3_pmi = finder3.score_ngrams(trigram_measures.pmi)

In [13]:
scored3_raw_freq = finder3.score_ngrams(trigram_measures.raw_freq)

In [ ]:
scored3_likelihood_ratio = finder3.score_ngrams(trigram_measures.likelihood_ratio)

In [14]:
quadgram_measures = nltk.collocations.QuadgramAssocMeasures()
finder4 = nltk.collocations.QuadgramCollocationFinder.from_documents(tokenized_reviews)
# здесь ничего, потому что квадграммы -- это вообще редкие штуки

KeyboardInterrupt: ignored

In [ ]:
scored4_pmi = finder4.score_ngrams(quadgram_measures.pmi)

In [ ]:
scored4_raw_freq = finder4.score_ngrams(quadgram_measures.raw_freq)

In [ ]:
scored4_likelihood_ratio = finder4.score_ngrams(quadgram_measures.likelihood_ratio)

In [ ]:
len(scored4_raw_freq)

Пробуем:

In [8]:
with open('final_ner.txt', 'r', encoding='utf-8') as f:
    final_ner = f.read().split('\n')

In [9]:
len(final_ner)

84412

In [10]:
final_ner_unigrams = []
final_ner_bigrams = []
final_ner_trigrams = []

for ne in final_ner:
    splited_ne = ne.split()
    if len(splited_ne) == 1:
        final_ner_unigrams.append(ne)
    if len(splited_ne) == 2:
        final_ner_bigrams.append(ne)
    if len(splited_ne) == 3:
        final_ner_trigrams.append(ne)

In [11]:
len(final_ner_unigrams)

8779

In [12]:
final_ner_bigrams[:10]

['town pure',
 'odd assortment',
 'ease thousand',
 'seated adult',
 'axis remarkable',
 'broad adult',
 'chair eleven',
 'recommendation unclear',
 'lean bongo',
 'yamahas yamahas']

In [14]:
def colls_for_unigrams(scored, ner_unigrams):
    '''
    Get bigrams with target unigrams.
    '''
    for_unigrams = {}
    for unigram in tqdm.tqdm(ner_unigrams):
        bigrams = []
        for bigram, score in scored:
            if unigram in bigram:
                bigram = ' '.join(bigram)
                bigrams.append((bigram, score))
        for_unigrams[unigram] = bigrams

    return for_unigrams

In [15]:
def colls_for_bigrams(scored, ner_bigrams):
    '''
    Get trigrams with target bigrams.
    '''
    for_bigrams = defaultdict(dict)
    for bigram in tqdm.tqdm(ner_bigrams):
        trigrams = []
        for trigram, score in scored:
            trigram = ' '.join(trigram)
            start_bigram = bigram + ' '
            end_bigram = ' ' + bigram
            if start_bigram in trigram or end_bigram in trigram:
                trigrams.append((trigram, score))
        for_bigrams[bigram] = trigrams
    
    return for_bigrams

In [16]:
def colls_for_trigrams(scored, ner_trigrams):
    '''
    Get quadgrams with target bigrams.
    '''
    for_trigrams = defaultdict(dict)
    for trigram in tqdm.tqdm(ner_trigrams):
        quadgrams = []
        for quadgram, score in scored:
            quadgram = ' '.join(quadgram)
            if trigram in quadgram or trigram in quadgram:
                quadgrams.append((quadgram, score))
        for_trigrams[trigram] = quadgrams

    return for_trigrams

In [ ]:
del model

Так как достать все ранжированные н-граммы достаточно трудно, мы возьмём по 5 униграмм, биграмм и триграмм для каждой метрики и посмотрим (на глаз), насколько хорошо они ранжируют коллокации.

* PMI:

In [ ]:
len(scored2_pmi)

674251

In [ ]:
bigrams_pmi = colls_for_unigrams(scored2_pmi, final_ner_unigrams[:5])

100%|██████████| 5/5 [00:02<00:00,  1.79it/s]


In [ ]:
del scored2_pmi

In [ ]:
bigrams_pmi

{'tidy': [('neat tidy', 11.125048199554987),
  ('tidy appearance', 8.685026939040675),
  ('tidy square', 8.579232275018079),
  ('fold tidy', 6.755729930383879),
  ('tidy setup', 6.109153412181275),
  ('tidy inside', 5.771877352960477),
  ('less tidy', 5.210844869038059),
  ('thing tidy', 5.06567955039802),
  ('everything tidy', 5.060112312919411),
  ('help tidy', 4.964225059977178),
  ('clean tidy', 4.473371373074354),
  ('keep tidy', 4.4419495663050625),
  ('nice tidy', 3.9896140379744196),
  ('really tidy', 3.6065860315602443),
  ('tidy case', 3.442556981982026),
  ('look tidy', 3.314579590785904),
  ('tidy thing', 3.2583246283404144),
  ('tidy pedal', 3.0094073463337203),
  ('tidy little', 2.8021181265684056),
  ('make tidy', 2.0886102854428117),
  ('tidy use', 1.6219679981506907)],
 'integrate': [('seamlessly integrate', 10.3970052595849),
  ('integrate seamlessly', 9.812042758863742),
  ('integrate logic', 7.535711530416492),
  ('integrate accept', 7.312940994796424),
  ('integrat

In [ ]:
trigrams_pmi = colls_for_bigrams(scored3_pmi, final_ner_bigrams[:5])

100%|██████████| 5/5 [00:04<00:00,  1.20it/s]


In [ ]:
trigrams_pmi

defaultdict(dict,
            {'town pure': [],
             'odd assortment': [],
             'ease thousand': [],
             'seated adult': [('height seated adult', 21.02980376220841),
              ('seated adult use', 15.622428494680204)],
             'axis remarkable': []})

In [ ]:
del scored3_pmi

Может ли так быть, что мы при обрезке по частотности вырезали вот такие биграммы?

In [ ]:
quadgrams_pmi = colls_for_trigrams(scored4_pmi, final_ner_trigrams[:5])

100%|██████████| 5/5 [00:37<00:00,  7.48s/it]


In [ ]:
quadgrams_pmi

defaultdict(dict,
            {'favorite capo tension': [('favorite capo tension adjustment',
               21.29915322344769)],
             'harmonica thast able': [('shape harmonica thast able',
               32.364048547832155),
              ('harmonica thast able fit', 29.504167722590346)],
             'attempt stre guitar': [('first attempt stre guitar',
               20.48175500815978),
              ('attempt stre guitar seem', 20.34699964668369)],
             'add assemble yamaha': [('add assemble yamaha keyboard',
               20.937746308544995),
              ('easy add assemble yamaha', 19.031223325383593)],
             'moisture yuk sax': [('moisture yuk sax sorry',
               35.99162531989234),
              ('keep moisture yuk sax', 31.7857009455742)]})

* Raw frequency

In [ ]:
bigrams_raw_freq = colls_for_unigrams(scored2_raw_freq, final_ner_unigrams[:5])

bigrams_raw_freq

100%|██████████| 5/5 [00:02<00:00,  2.34it/s]


{'tidy': [('neat tidy', 1.881217605071068e-06),
  ('thing tidy', 1.0129633258074982e-06),
  ('nice tidy', 7.235452327196415e-07),
  ('really tidy', 5.788361861757132e-07),
  ('keep tidy', 4.341271396317849e-07),
  ('less tidy', 4.341271396317849e-07),
  ('look tidy', 4.341271396317849e-07),
  ('tidy pedal', 4.341271396317849e-07),
  ('tidy setup', 4.341271396317849e-07),
  ('tidy use', 4.341271396317849e-07),
  ('clean tidy', 2.894180930878566e-07),
  ('everything tidy', 2.894180930878566e-07),
  ('fold tidy', 2.894180930878566e-07),
  ('help tidy', 2.894180930878566e-07),
  ('make tidy', 2.894180930878566e-07),
  ('tidy appearance', 2.894180930878566e-07),
  ('tidy case', 2.894180930878566e-07),
  ('tidy inside', 2.894180930878566e-07),
  ('tidy little', 2.894180930878566e-07),
  ('tidy square', 2.894180930878566e-07),
  ('tidy thing', 2.894180930878566e-07)],
 'integrate': [('integrate well', 1.5917995119832115e-06),
  ('integrate computer', 1.0129633258074982e-06),
  ('easy integrat

In [ ]:
trigrams_raw_freq = colls_for_bigrams(scored3_raw_freq, final_ner_bigrams[:5])

trigrams_raw_freq

100%|██████████| 5/5 [00:07<00:00,  1.55s/it]


defaultdict(dict,
            {'town pure': [],
             'odd assortment': [],
             'ease thousand': [],
             'seated adult': [('height seated adult', 2.894180930878566e-07),
              ('seated adult use', 2.894180930878566e-07)],
             'axis remarkable': []})

In [ ]:
quadgrams_raw_freq = colls_for_trigrams(scored4_raw_freq, final_ner_trigrams[:5])

quadgrams_raw_freq

100%|██████████| 5/5 [00:33<00:00,  6.60s/it]


defaultdict(dict,
            {'favorite capo tension': [('favorite capo tension adjustment',
               1.447090465439283e-07)],
             'harmonica thast able': [('harmonica thast able fit',
               1.447090465439283e-07),
              ('shape harmonica thast able', 1.447090465439283e-07)],
             'attempt stre guitar': [('attempt stre guitar seem',
               1.447090465439283e-07),
              ('first attempt stre guitar', 1.447090465439283e-07)],
             'add assemble yamaha': [('add assemble yamaha keyboard',
               1.447090465439283e-07),
              ('easy add assemble yamaha', 1.447090465439283e-07)],
             'moisture yuk sax': [('keep moisture yuk sax',
               1.447090465439283e-07),
              ('moisture yuk sax sorry', 1.447090465439283e-07)]})

* Likelihood ratio

In [ ]:
bigrams_likelihood_ratio = colls_for_unigrams(scored2_likelihood_ratio, final_ner_unigrams[:5])

bigrams_likelihood_ratio

100%|██████████| 5/5 [00:02<00:00,  2.01it/s]


{'tidy': [('neat tidy', 177.27837961666233),
  ('thing tidy', 36.2322647311642),
  ('tidy appearance', 20.15342900389465),
  ('tidy square', 19.860206599167498),
  ('tidy setup', 19.617764873094305),
  ('nice tidy', 18.5928670402835),
  ('less tidy', 15.952853579558479),
  ('fold tidy', 14.82441085695887),
  ('keep tidy', 12.86414730320745),
  ('really tidy', 12.845053048815561),
  ('tidy inside', 12.130676313295501),
  ('everything tidy', 10.202237773974474),
  ('help tidy', 9.944357364858696),
  ('clean tidy', 8.633794025837709),
  ('look tidy', 8.489695406511537),
  ('tidy pedal', 7.357315491781047),
  ('tidy case', 5.9585728554303525),
  ('tidy thing', 5.496618048761131),
  ('tidy little', 4.38342403656304),
  ('make tidy', 2.76383370282236),
  ('tidy use', 2.752464938512066)],
 'integrate': [('integrate computer', 46.00182199538949),
  ('integrate daw', 39.93944176816546),
  ('seamlessly integrate', 37.40988833826202),
  ('integrate system', 26.25897163410709),
  ('integrate logic

In [ ]:
trigrams_likelihood_ratio = colls_for_bigrams(scored3_likelihood_ratio, final_ner_bigrams[:5])

trigrams_likelihood_ratio

100%|██████████| 5/5 [00:04<00:00,  1.16it/s]


defaultdict(dict,
            {'town pure': [],
             'odd assortment': [],
             'ease thousand': [],
             'seated adult': [('height seated adult', 63.162594277103494),
              ('seated adult use', 42.710783163812934)],
             'axis remarkable': []})

In [ ]:
quadgrams_likelihood_ratio = colls_for_trigrams(scored4_likelihood_ratio, final_ner_trigrams[:5])

quadgrams_likelihood_ratio

100%|██████████| 5/5 [00:36<00:00,  7.23s/it]


defaultdict(dict,
            {'favorite capo tension': [('favorite capo tension adjustment',
               353.43786653372365)],
             'harmonica thast able': [('harmonica thast able fit',
               179.7447730096226),
              ('shape harmonica thast able', 54.443633897871216)],
             'attempt stre guitar': [('first attempt stre guitar',
               423.7871538222854),
              ('attempt stre guitar seem', 163.5662340549925)],
             'add assemble yamaha': [('add assemble yamaha keyboard',
               938.7082555770414),
              ('easy add assemble yamaha', 64.19002334196469)],
             'moisture yuk sax': [('keep moisture yuk sax',
               143.61413154187193),
              ('moisture yuk sax sorry', 68.63110066606251)]})

Попробуем взять raw frequency модель

In [18]:
bigrams_raw_freq = colls_for_bigrams(scored2_raw_freq, final_ner_unigrams[:5])
for key, value in bigrams_raw_freq.items():
    print('\n')
    print(key)
    print('-----')
    for v in value[:5]:
        print(v[0])

100%|██████████| 5/5 [00:02<00:00,  1.83it/s]



tidy
-----
neat tidy
thing tidy
nice tidy
really tidy
keep tidy


integrate
-----
integrate well
integrate computer
easy integrate
integrate guitar
integrate tuner


dandy
-----
jim dandy
fine dandy
handy dandy
dandy little
dandy 's


importantly
-----
importantly sound
perhaps importantly
use importantly
importantly fit
importantly one


pineapple
-----
soprano pineapple
pineapple shape
pineapple soprano
pineapple ukulele
kala pineapple


## Объединение

Мне кажется, хороший способ объединить несколько NE под одним лейблом -- это снова провекторизовать их, кластеризовать и выбрать центроиды кластеров; после того как центроиды кластеров будут выбраны, мы собираем все коллокации, принадлежащие NE, которые вошли в кластер, под одним названием.

Другой базовый способ -- это поиск с помощью косинусного сходства. Мы векторизуем NE (и возможно коллокации) и считаем сходство между ними. Для каждого выбираем ближайших соседей; если есть пересечение, то объединяем их под одним лейблом.